# Deploying and invoking Drift Detector Dataflow Flex template

In [1]:
%cd drift_detector

/home/jupyter/drift-monitor/drift_detector


In [2]:
import json
from datetime import datetime
import time

## Build the Flex template docker image

In [3]:
template_name = 'drift-detector'

project_id = !(gcloud config get-value core/project)
project_id = project_id[0]
template_image = 'gcr.io/{}/{}:latest'.format(project_id, template_name)
location = 'us-central1'

In [4]:
!gcloud builds submit --tag {template_image} .

Creating temporary tarball archive of 13 file(s) totalling 42.5 KiB before compression.
Uploading tarball of [.] to [gs://mlops-dev-env_cloudbuild/source/1590128151.63-63c6732b38a44f1abac58ac9b14e5cf9.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mlops-dev-env/builds/b69c309b-7dab-4cf1-864e-75556269795b].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/b69c309b-7dab-4cf1-864e-75556269795b?project=881178567352].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "b69c309b-7dab-4cf1-864e-75556269795b"

FETCHSOURCE
Fetching storage object: gs://mlops-dev-env_cloudbuild/source/1590128151.63-63c6732b38a44f1abac58ac9b14e5cf9.tgz#1590128152075220
Copying gs://mlops-dev-env_cloudbuild/source/1590128151.63-63c6732b38a44f1abac58ac9b14e5cf9.tgz#1590128152075220...
/ [1 files][ 10.5 KiB/ 10.5 KiB]                                                
Operation completed over 1 objects/10.5 KiB.                         

## Deploy the Flex template using `gcloud`

In [5]:
flex_templates_location = 'gs://mlops-dev-workspace/flex-templates'
template_path = '{}/{}.json'.format(flex_templates_location, template_name)
metadata_file = 'metadata.json'

In [6]:
!gcloud beta dataflow flex-template build {template_path} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file}

Successfully saved container spec in flex template file.
Template File GCS Location: gs://mlops-dev-workspace/flex-templates/drift-detector.json
Container Spec:

{
    "image": "gcr.io/mlops-dev-env/drift-detector:latest",
    "metadata": {
        "description": "Data drift detector Python flex template.",
        "name": "Data drift detector Python flex template",
        "parameters": [
            {
                "helpText": "A full name of the BQ request-response log table",
                "label": "Request response log table.",
                "name": "request_response_log_table",
                "regexes": [
                    "[-_.a-zA-Z0-9]+"
                ]
            },
            {
                "helpText": "A type of instances in request_response log_table",
                "label": "Instance type.",
                "name": "instance_type",
                "regexes": [
                    "LIST|OBJECT"
                ]
            },
            {
              

## Trigger a run of the template using REST API

In [7]:
import googleapiclient.discovery

service = googleapiclient.discovery.build('dataflow', 'v1b3')

parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'instance_type': 'OBJECT',
    'start_time': '2020-05-15T00:15:00',
    'end_time': '2020-05-15T05:51:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf/test1',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'setup_file': './setup.py',

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

request = service.projects().locations().flexTemplates().launch(
    location=location,
    projectId=project_id,
    body=body)

response = request.execute()
response

{'job': {'id': '2020-05-21_23_18_23-11550106382458589032',
  'projectId': 'mlops-dev-env',
  'name': 'data-drift-20200522-061823',
  'currentStateTime': '1970-01-01T00:00:00Z',
  'createTime': '2020-05-22T06:18:24.491636Z',
  'location': 'us-central1',
  'startTime': '2020-05-22T06:18:24.491636Z'}}

## Trigger a run of the template using REST API - Curl

In [8]:
service_url = 'https://dataflow.googleapis.com/v1b3/projects/{}/locations/us-central1/flexTemplates:launch'.format(project_id)
headers_content = 'Content-Type: application/json'
access_token = !(gcloud auth print-access-token) 
headers_auth = 'Authorization: Bearer {}'.format(access_token[0])
parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'instance_type': 'OBJECT',
    'start_time': '2020-05-15T00:15:00',
    'end_time': '2020-05-15T05:51:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf/test2',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'setup_file': './setup.py',

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

json_body = json.dumps(body)

In [9]:
!curl -X POST \
  "{service_url}" \
  -H "{headers_content}" \
  -H "{headers_auth}" \
  -d '{json_body}'

{
  "job": {
    "id": "2020-05-21_23_20_21-12279682000606787233",
    "projectId": "mlops-dev-env",
    "name": "data-drift-20200522-062019",
    "currentStateTime": "1970-01-01T00:00:00Z",
    "createTime": "2020-05-22T06:20:22.748868Z",
    "location": "us-central1",
    "startTime": "2020-05-22T06:20:22.748868Z"
  }
}


## Trigger a run of the template using `gcloud`

In [ ]:
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
g_parameters = ','.join(['{}={}'.format(key,value) for key, value in parameters.items()])

!gcloud beta dataflow flex-template run {job_name} \
--template-file-gcs-location {template_path} \
--parameters {g_parameters}